In [16]:
import pandas as pd

Read jobOrder files that come from the cluster, group A is without cylinder and power axis to be customized, group B is with all parameters. File is the same as jobOrder but with a column added that has the cluster number

In [17]:
# Read Files
jobOrder_Cluster_A = pd.read_csv("clusterA_kmeans.csv",
                                 infer_datetime_format=True,
                                 parse_dates=[6])
jobOrder_Cluster_B = pd.read_csv("clusterB_kmeans.csv",
                                 infer_datetime_format=True,
                                 parse_dates=[7])

In [32]:
Proportion_A = jobOrder_Cluster_A.shape[0]/(jobOrder_Cluster_B.shape[0]+jobOrder_Cluster_A.shape[0])

In [33]:
Proportion_B = jobOrder_Cluster_B.shape[0]/(jobOrder_Cluster_B.shape[0]+jobOrder_Cluster_A.shape[0])

The following line predefines an amount to made to stock, but the final code will have as input whatever comes out of the Time Series part, so the final code here is missing.

In [21]:
Amount_to_Produce = pd.read_csv('Amount_To_Be_Produced.csv', index_col=0)
made_to_stock = -sum(Amount_to_Produce[Amount_to_Produce['Difference']<0]['Difference'].astype(int))
print(f'Amount to Made to Stock: {made_to_stock} units')

Amount to Made to Stock: 79005 units


We now create a column which concatenates all the parameters into one, separated by ",", then we can group by that column to get repeated parameter combinations.

In [23]:
jobOrder_Cluster_A['PARAMETER_COMBINATION'] = \
jobOrder_Cluster_A.apply(lambda row: ','.join([str(row['BASE_RADIUS']),
                                               str(row['POWER']),
                                               str(row['DIAMETER']),
                                               str(row['CENTER_THK'])]), axis=1)
jobOrder_Cluster_B['PARAMETER_COMBINATION'] = \
jobOrder_Cluster_B.apply(lambda row: ','.join([str(row['BASE_RADIUS']),
                                               str(row['POWER']),
                                               str(row['DIAMETER']),
                                               str(row['CENTER_THK']),
                                               str(row['CYLINDER']),
                                               str(row['POWER_AXIS'])]), axis=1)

In [28]:
# Create a year column
jobOrder_Cluster_A['Year'] = jobOrder_Cluster_A['LAUNCH_DATE'].dt.year
jobOrder_Cluster_B['Year'] = jobOrder_Cluster_B['LAUNCH_DATE'].dt.year

We create a function that selects the largest clusters that in sum amount for the total made to stock number. The ratio of production follows the logic that we are not producing the total amount of units for every combination, say if we predict 500 units of a combiantion will sell, and the ratio_production is 0.5, we only produce 250. The output is a list of clusters.

In [29]:
def get_selected_clusters(df, made_to_stock, ratio_production = 0.5):
    # Step 1: Group by 'cluster' and calculate the count of rows for each cluster
    cluster_counts = df.groupby('Cluster').size()

    # Step 2: Sort the groups by count in descending order
    sorted_clusters = cluster_counts.sort_values(ascending=False)

    # Step 3: Select the top clusters until the total count reaches or exceeds X
    selected_clusters = []
    count_sum = 0

    for cluster, count in sorted_clusters.items():
        count_sum += count
        selected_clusters.append(cluster)

        if count_sum >= made_to_stock/ratio_production:
            break
    return selected_clusters

In [30]:
jobOrder_Cluster_A[jobOrder_Cluster_A['Year'].isin([2020,2021,2022])]

,MATERIAL_CODE,BASE_RADIUS,POWER,DIAMETER,CENTER_THK,LAUNCH_DATE,CodFam,CodSubFam,month,Cluster,PARAMETER_COMBINATION,Year
PRIMARY_KEY,,,,,,,,,,,,
FO8973_1_0,3201.0,8.0,-3.50,14.5,0.1000,2020-01-02 10:07:00,1.0,5.0,1,12,"8.0,-3.5,14.5,0.1",2020
FO9775_2_0,3201.0,8.3,-1.25,14.5,0.1200,2020-01-02 05:57:00,1.0,5.0,1,7,"8.3,-1.25,14.5,0.12",2020
FO9799_2_0,3201.0,8.3,-2.25,14.5,0.1000,2020-01-02 05:25:00,1.0,5.0,1,7,"8.3,-2.25,14.5,0.1",2020
FO9799_3_0,3201.0,8.3,-2.25,14.5,0.1000,2020-01-02 05:57:00,1.0,5.0,1,7,"8.3,-2.25,14.5,0.1",2020
FO9958_2_0,3201.0,8.9,-3.75,15.0,0.1000,2020-01-02 09:25:00,1.0,5.0,1,7,"8.9,-3.75,15.0,0.1",2020
...,...,...,...,...,...,...,...,...,...,...,...,...
I38162_2_0,7401.0,8.6,4.75,14.0,0.2498,2022-12-30 20:32:00,1.0,1.0,12,16,"8.6,4.75,14.0,0.2498",2022
I38162_3_0,7401.0,8.6,4.75,14.0,0.2498,2022-12-30 20:32:00,1.0,1.0,12,16,"8.6,4.75,14.0,0.2498",2022
I38162_4_0,7401.0,8.6,4.25,14.0,0.2383,2022-12-30 20:32:00,1.0,1.0,12,16,"8.6,4.25,14.0,0.2383",2022


The following function, takes the jobOrder dataframe, gets the most frequent clusters of the last year (using the previous function), then filters the data only to show those clusters. After, it pivots the table to get the amount ordered of each combination over the last 3 years, then takes the average of the last 3 years, and calculates a made to stock number based on the average and the ratio_production, so it produces the average of the last 3 years multiplied by the ratio. The output is a dataframe showing the parameters combinations, their last 3 years orders, the average and the amount to make to stock. The minimum_production determines the minimum amount to produce of a single combination.

In [67]:
# Function for getting a dataframe with the combinations and units to make to stock
import math
def get_made_to_stock(df, made_to_stock, minimum_production = 10, ratio_production = 0.5):
    
    # Obtain clusters that add up to the target made to stock units
    clusters = get_selected_clusters(df[df['Year']==2022], made_to_stock, ratio_production)
    
    # Filter dataframe by year and clusters
    df = df[df['Year'].isin([2020,2021,2022])]
    df = df[df['Cluster'].isin(clusters)]
    
    # Pivot table per combination and year, get the minimum of the last 3 years, order and filter by minimum production.
    df = df.pivot_table(index='PARAMETER_COMBINATION', columns='Year', values='PRIMARY_KEY', aggfunc='count').fillna(0)
    df['AVG_LAST3'] = df.mean(axis=1)
    df.sort_values(by='AVG_LAST3', ascending=False, inplace=True)
    df = df[df['AVG_LAST3']>minimum_production]
    
    # Redefine target ratio to add up to the made to stock units
    target_ratio = made_to_stock/sum(df['AVG_LAST3'])
    
    # Calculate made to stock of each combination, rounding down
    if target_ratio < 1:
        df['MADE_TO_STOCK'] = df['AVG_LAST3']*target_ratio
    else:
        df['MADE_TO_STOCK'] = df['AVG_LAST3']
    df['MADE_TO_STOCK'] = df['MADE_TO_STOCK'].astype(int)
    return df

In [68]:
made_to_stock_orders_A = get_made_to_stock(jobOrder_Cluster_A, made_to_stock*Proportion_A)
made_to_stock_orders_B = get_made_to_stock(jobOrder_Cluster_B, made_to_stock*Proportion_B)

In [74]:
made_to_stock_orders_A

Year,2020,2021,2022,AVG_LAST3,MADE_TO_STOCK
PARAMETER_COMBINATION,,,,,
"8.3,-3.5,14.5,0.12",84.0,363.0,509.0,318.666667,246
"8.3,-2.5,14.5,0.12",146.0,311.0,483.0,313.333333,242
"8.3,-3.25,14.5,0.12",87.0,361.0,404.0,284.000000,219
"8.3,-3.0,14.5,0.12",128.0,287.0,411.0,275.333333,213
"8.3,-2.0,14.5,0.12",90.0,292.0,441.0,274.333333,212
...,...,...,...,...,...
"8.6,-4.0,15.0,0.1",31.0,0.0,0.0,10.333333,7
"8.6,0.0,14.5,0.1515",0.0,0.0,31.0,10.333333,7
"8.3,-10.0,15.0,0.12",1.0,26.0,4.0,10.333333,7


In [73]:
made_to_stock_orders_B

Year,2020,2021,2022,AVG_LAST3,MADE_TO_STOCK
PARAMETER_COMBINATION,,,,,
"8.7,-8.0,14.4,0.1101,-1.25,20.0",31.0,32.0,45.0,36.000000,36
"8.7,-6.0,14.4,0.1101,-0.75,10.0",20.0,34.0,36.0,30.000000,30
"8.7,-5.0,14.4,0.1589,-0.75,20.0",34.0,30.0,25.0,29.666667,29
"8.6,-8.0,14.5,0.1401,-2.25,30.0",42.0,22.0,24.0,29.333333,29
"8.7,-6.0,14.4,0.1101,-1.75,10.0",21.0,20.0,38.0,26.333333,26
...,...,...,...,...,...
"8.7,-2.75,14.4,0.2118,-0.75,40.0",12.0,12.0,7.0,10.333333,10
"8.7,-1.0,14.4,0.1974,-1.25,10.0",11.0,7.0,13.0,10.333333,10
"8.7,-3.0,14.4,0.1969,-1.25,30.0",6.0,12.0,13.0,10.333333,10
